In [8]:
import numpy as np
import pandas as pd
import pickle
import time
from numpy import genfromtxt
from scipy.sparse import coo_matrix
from scipy.sparse import save_npz

<br> 
1. Load dictionary of active pages
2. Load page categories and keep only those for the pages with the active users.

In [2]:
g = open("page_categories.pkl", "rb")
page_categories = pickle.load(g)
g.close()

f = open("C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_page_likes.pkl", "rb")
temp = pickle.load(f)
f.close()

active_page_ids = []

for key in temp.keys():
    active_page_ids.append(key)

active_page_ids = sorted(active_page_ids)

active_page_categories = {}

for key in page_categories.keys():
    if key in active_page_ids:
        active_page_categories[key] = page_categories[key]

<br>
<span>3.</span> Make a dictionary of page categories leaving Category 7 out. 

In [6]:
active_page_categories_ex_7 = {}

for key in active_page_categories.keys():
    if active_page_categories[key] != 7:
        active_page_categories_ex_7[key] = active_page_categories[key]

In [10]:
print("There are", len(active_page_categories_ex_7), "pages left without category 7. ")

There are 5911 pages left without category 7. 


In [16]:
f = open("C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_page_likes.pkl", "rb")
temp = pickle.load(f)
f.close()

In [17]:
active_users_page_likes_ex_7 = {}

for key in temp.keys():
    if str(key) in active_page_categories_ex_7.keys():
        active_users_page_likes_ex_7[key] = temp[key]

In [21]:
len(active_users_page_likes_ex_7)

5911

In [22]:
f = open("C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_page_likes_ex_7.pkl","wb") # dict{page: users with at least 2 click}
pickle.dump(active_users_page_likes_ex_7,f)
f.close()

<br>
<span>4.</span> Build the new sparse matrix. 

In [3]:
f = open("C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_page_likes_ex_7.pkl", "rb")
temp = pickle.load(f)
f.close()


d = sorted(temp.keys())

active_users_page_likes_ex_7 = {}

for i in range(0, len(d)):
    key = d[i]
    active_users_page_likes_ex_7[key] = temp[key]

In [4]:
# Build the sparse matrix with the number of common likers. Users should have liked both pages at least twice. 

r = 0 # the index of the row of the matrix
c = 0 # the index of the columns of the matrix
row_indices = []
column_indices = []
matrix_elements = []

t1 = time.time()

for key_r in active_users_page_likes_ex_7.keys(): # key_r is the key for the rows
    x = active_users_page_likes_ex_7[key_r]
    for key_c in active_users_page_likes_ex_7.keys(): # key_c is the key for the columns
        if c >= r:
            y = active_users_page_likes_ex_7[key_c]
            common_set = set(x)&set(y)
            common_set_size = len(common_set)
            if common_set_size > 0:
                row_indices.append(r)
                column_indices.append(c)
                matrix_elements.append(common_set_size)
        c = c + 1
    r = r + 1
    c = 0

print(round((time.time() - t1) / 3600,2), "hours")

np.savetxt('C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_ex_7_mx_row_indices.csv', row_indices, delimiter=',')
np.savetxt('C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_ex_7_mx_column_indices.csv', column_indices, delimiter=',')
np.savetxt('C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_ex_7_mx_elements.csv', matrix_elements, delimiter=',')

3.17 hours


In [2]:
row_indices = np.loadtxt('C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_ex_7_mx_row_indices.csv', delimiter=',')
column_indices = np.loadtxt('C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_ex_7_mx_column_indices.csv', delimiter=',')
matrix_elements = np.loadtxt('C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_ex_7_mx_elements.csv', delimiter=',')

In [3]:
row_indices = row_indices.tolist()
column_indices = column_indices.tolist()
matrix_elements = matrix_elements.tolist()

row_indices_2 = row_indices
column_indices_2 = column_indices
matrix_elements_2 = matrix_elements

for i in range(0, len(row_indices)):
    if row_indices[i] != column_indices[i]:
        column_indices_2.append(row_indices[i])
        row_indices_2.append(column_indices[i])
        matrix_elements_2.append(matrix_elements[i])

In [4]:
size = 5911

row2  = np.array(row_indices_2)
col2  = np.array(column_indices_2)
val2 = np.array(matrix_elements_2)
mx = coo_matrix((val2, (row2, col2)), shape=(size, size))

In [6]:
print("Matrix fill-up =", round(len(val2) / 5911**2, 3), ".")

Matrix fill-up = 0.349 .


In [9]:
save_npz('C:\\FOLDERS\\CEU\\Capstone Project\\Data\\active_users_sparse_matrix_ex_7.npz', mx)